In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import concurrent.futures
import time

### Functions that Will Extract Location and Address Data

In [ ]:
##this function extracts the location (City State and Zipcode) for each school.

def extractLocation(soup):
    address_element = soup.select_one('address') 
        #print(address_element)
        # Exclude the text within the span within the address element
    if address_element:
        city_state = address_element.find('span')
    
    else:
        #print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None  # Return None or some default value if the request fails

    return city_state.text

In [ ]:
#this function extracts the address for each school.

def extractAddress(soup):
        # Find the address element
    address_element = soup.select_one('address') 
        #print(address_element)
        # Exclude the text within the span within the address element

    city_state = address_element.find('span')
    city_state.decompose()

    address_text = address_element.get_text(strip=True)
    if address_text != '':
        return address_text
    else:
        return None

In [ ]:
#this function cleans up the location column by splitting it into city, state and zipcode columns.
def splitLocation(column):
    
    pattern = r'(?P<city>[^,]+).\s?(?P<state>\w\w)\s?(?P<zipcode>\d\d\d\d\d)?'
    #column = pd.Series(column)
    _city = column.str.extract(pattern)['city']
    _state = column.str.extract(pattern)['state']
    _zip = column.str.extract(pattern)['zipcode']
    
    return (_city, _state, _zip)

### Functions that extract the Score and Game info

In [ ]:
#this funciton cleans up the gameinfo column by splitting it into venue, team2 and gametype (H or A) columns.
def gameInfo(column):
    pattern = r'(?P<venue>[vs@]*)(?P<name>[a-zA-Z]+(\.|-)?\s?[a-zA-Z]*\'?\w*)(?P<type>[*]*)'
    # Apply str.extract to create new columns based on the pattern
    _venue = column.str.extract(pattern)['venue']
    _team2 = column.str.extract(pattern)['name']
    _gametype = column.str.extract(pattern)['type']
    
    
    return (_venue, _team2, _gametype)

In [ ]:
#this function cleans up the score column by splitting it into team, team1_score and team2_score columns.
def scoreInfo(column):
    # Define a regular expression pattern to extract W or L, and the scores
    pattern = r'([WL])(\d+)-(\d+)|([WL])\(FF\)'

    # Extract the components into separate columns
    extracted = column.str.extract(pattern)

    # Assign extracted values to new columns
    team = extracted[0].fillna(extracted[3])
    team1_score = extracted[1]#.fillna(pd.NA).astype('Int64')
    team2_score = extracted[2]#.fillna(pd.NA).astype('Int64')

    return team, team1_score, team2_score

### Functions that format the game and venue type correctly

In [ ]:
#each game entry will have  *, **, or *** next to it which correponds to the game type. This function cleans up the game type column.
def cleanGameType(string):
    #string = str(len(string))
    if len(string) == 3:
        string = 'Tournament'
    elif len(string) == 2:
        string = 'Playoff'
    elif len(string) == 1:
        string = 'District'
    else:
        string = 'Regular Season'
    return string

In [ ]:
def cleanVenueType(string):
    if string == '@':
        return 'A'
    return 'H'

### Making a Dict with each school's name and URL

In [ ]:
#this function extracts the number of pages on Maxpreps for each state.
def get_num_pages(state):
    current_page = 1
    while True:
        url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        next_page_button = soup.find_all('a', class_ = 'btn btn-default')
        button = [butt.text for butt in next_page_button]
        num = str(current_page + 1)
        if num not in button:
            return int(num)- 1
        current_page += 1

In [ ]:
#this function will extract 
def extract_state_ranking_page(state, current_page):
    url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text , 'html.parser')
    school_elements = soup.find_all('th', class_='school')
    school_elements.pop(0)
    school_links = [school.find('a').get('href') for school in school_elements]
    return dict(zip((school.text for school in school_elements), school_links))

In [ ]:
#this function will create a dictionary of schools and their respective links given a state.
def create_school_dict(state):
    school_dict = {}
    num_pages = get_num_pages(state)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_page = {executor.submit(extract_state_ranking_page, state, page): page for page in range(1, num_pages + 1)}
        for future in concurrent.futures.as_completed(future_to_page):
            try:
                page = future_to_page[future]
                data = future.result()
                school_dict.update(data)
            except Exception as e:
                print(f"Error fetching page {page}: {e}")
    return school_dict

### Team 2 URL Extraction

In [ ]:
def extractUrls(soup):
    team2_urls = []
    
    # Find all 'tr' elements with the specific class
    rows = soup.find_all('tr', class_='sc-f64836fa-0 sc-3dbc5bfc-0 dOVNcf hCLLbb')
    
    # Loop through each row and find the 'a' tag with the given class
    for row in rows:
        link = row.find('a', class_='sc-ed4e19a6-0 cYQoLo')
        
        # If the 'a' tag is found, extract the href, otherwise append None
        if link:
            team2_urls.append(link.get('href'))
        else:
            team2_urls.append(None)  # Or '' if you prefer an empty string

    return team2_urls


### Sports Offered Extraction

In [ ]:
#this function extracts the sports offered at a given school for both boys and girls. It takes in the school URL as input
def extractSports(url):
    sports_url = f'http://www.maxpreps.com{url}'
    response = requests.get(sports_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    elements = soup.find_all('div', class_ = 'sports-list-child')

    boys = []
    girls = []
    #check if school offers no sports
    if len(elements) == 0:
        return None

    for element in elements:
        #gender = element.find('h2', class_ = 'sc-23c90da8-0 KWggY')
        gender = element.find('h2', class_ = 'sc-5236d688-0 cvMzgp')
        sports = element.find_all('span', class_ = 'sport-name')
        if gender.text == 'Boys':
            for sport in sports:
                boys.append(sport.text)
        else:
            for sport in sports:
                girls.append(sport.text)
                
    return ','.join(boys), ','.join(girls)

### Put it all together

In [ ]:
import concurrent.futures
from tenacity import retry, stop_after_attempt, wait_fixed
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_after_attempt, wait_fixed

In [ ]:
#this function utilizes all previous functions. it will read in the school name and url, scrape the data, and clean everything up to a useable format.
from io import StringIO
@retry(stop=stop_after_attempt(5), wait=wait_fixed(3))
def scrape_school_data(school, url):
    try:
        school_url = f'http://www.maxpreps.com{url}'
        response = requests.get(school_url).text
        soup = BeautifulSoup(response, 'html.parser')

        table = soup.find('table')
        html_table = str(table)
        data = pd.read_html(StringIO(html_table))[0]
        
        data = data.drop('Game Info', axis=1)
        data['Team 1'] = school
        data['Team 1 Location'] = extractLocation(soup)
        data['Team 1 Address'] = extractAddress(soup)
        data['Team 1 City'] = splitLocation(data['Team 1 Location'])[0]
        data['Team 1 State'] = splitLocation(data['Team 1 Location'])[1]
        data['Team 1 Zipcode'] = splitLocation(data['Team 1 Location'])[2]
        data['Venue'] = gameInfo(data['Opponent'])[0].apply(cleanVenueType)
        data['Team 2'] = gameInfo(data['Opponent'])[1]
        data['Game Type'] = gameInfo(data['Opponent'])[2].apply(cleanGameType)
        data['Outcome'] = scoreInfo(data['Result'])[0]
        data['Team 1 Score'] = scoreInfo(data['Result'])[1]
        data['Team 2 Score'] = scoreInfo(data['Result'])[2]
        data['Team 2 URL'] = extractUrls(soup)
        sports_url = url.rsplit('/', 4)[0] + '/'
        sports_offered = extractSports(sports_url)
        data['Boys Sports'] = sports_offered[0]
        data['Girls Sports'] = sports_offered[1]
        data['Team 1 URL'] = url
        
        return data

    except Exception as e:
        print(f"Error processing {school}: {e}")
        return None

In [ ]:
#this function is given a state and a dictionary of schools and their urls. It will scrape all the schools in the dictionary and return a dataframe with all the data.
def scrape_all_schools_async(school_dict, state):

    grand_df = pd.DataFrame()

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(scrape_school_data, school, url) for school, url in school_dict.items()]

        with tqdm(total=len(futures), desc=f"Scraping Schools for {state}", unit=" schools") as pbar:
            for future in concurrent.futures.as_completed(futures):
                data = future.result()
                if data is not None:
                    grand_df = pd.concat([grand_df, data], ignore_index=True)
                pbar.update(1)

    return grand_df

### Running It

In [ ]:
states = [
    "ak", "al", "ar", "az", "ca", "co", "ct", "dc", "de", "fl", "ga",
    "hi", "ia", "id", "il", "in", "ks", "ky", "la", "ma", "md",
    "me", "mi", "mn", "mo", "ms", "mt", "nc", "nd", "ne", "nh",
    "nj", "nm", "nv", "ny", "oh", "ok", "or", "pa", "ri", "sc",
    "sd", "tn", "tx", "ut", "va", "vt", "wa", "wi", "wv", "wy"
]

#this loop:
#(1) iterates through each state
#(2) in each iteration a school dictionary is created with school as key and url as value
#(3) the data is scraped for each school in the dictionary
#(4) Team 2 Location and Address is added using the mergeDF function
#(5) columns are reordered
#(6) the data for each state is saved in its own csv file
for state in states:
    school_dict = create_school_dict(state)
    school_dict = {k: school_dict[k] for k in sorted(school_dict)}
    #print(school_dict)
    data = scrape_all_schools_async(school_dict, state)
    cols = ['Date', 'Team 1', 'Team 2', 'Venue', 'Game Type', 'Team 1 Score', 
       'Team 2 Score', 'Outcome', 'Team 1 Address', 'Team 1 City',
       'Team 1 State', 'Team 1 Zipcode','Team 1 URL', 'Team 2 URL', 'Boys Sports', 'Girls Sports']
    data = data[cols]
    data.to_csv(f'{state}.csv', index=False)

### Merge 50 States Data

In [ ]:
import pandas
import os
import glob

In [ ]:
#checking the current directory that we are im, change to the one that contains the csv files for all 50 states if nessesary
os.getcwd()
os.chdir('/Users/rdhir/Documents/maxpreps-data-analysis/Data/Individual States')

In [ ]:
ls

In [ ]:
#itereate through each csv file within the directory then read and concat them into a grand df
folder_path = '/Users/rdhir/Documents/maxpreps-data-analysis/Data/Individual States/*csv'
filelist = glob.glob(folder_path)

dfs = []
for file in filelist:
    df = pd.read_csv(file)
    dfs.append(df)

all_states = pd.concat(dfs, ignore_index=True)


In [200]:
all_states

,Date,Team 1,Team 2,Venue,Game Type,Team 1 Score,Team 2 Score,Outcome,Team 1 Address,Team 1 City,Team 1 State,Team 1 Zipcode,Team 1 URL,Team 2 URL,Boys Sports,Girls Sports
0,12/2,Academy of Holy Angels (Richfield),Edison,H,Regular Season,72.0,62.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/minneapolis/edison-tommies/basketball/21-2...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball..."
1,12/7,Academy of Holy Angels (Richfield),Minneapolis Southwest,A,Regular Season,62.0,37.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/minneapolis/minneapolis-southwest-lakers/b...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball..."
2,12/9,Academy of Holy Angels (Richfield),Highland Park,A,Regular Season,65.0,50.0,W,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/st-paul/highland-park-scots/basketball/21-...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball..."
3,12/14,Academy of Holy Angels (Richfield),Two Rivers,H,Regular Season,74.0,69.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/mendota-heights/two-rivers-warriors/basket...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball..."
4,12/16,Academy of Holy Angels (Richfield),St. Thomas,H,Regular Season,74.0,70.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/mendota-heights/st-thomas-academy-cadets/b...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468312,2/10,White Sulphur Springs,West Yellowstone,H,District,77.0,54.0,L,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/yellowstone/west-yellowstone-wolverines/ba...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball"
468313,2/12,White Sulphur Springs,Shields Valley,H,District,91.0,59.0,L,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/clyde-park/shields-valley-rebels/basketbal...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball"
468314,2/16,White Sulphur Springs,Lone Peak,H,Playoff,71.0,60.0,W,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/gallatin-gateway/lone-peak-big-horns/baske...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball"
468315,2/17,White Sulphur Springs,Shields Valley,H,Playoff,74.0,48.0,L,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/clyde-park/shields-valley-rebels/basketbal...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball"


### Add Strength of Schedule, State Ranking, and Rating data

In [ ]:
#this function extracts the number of pages on Maxpreps for each state.
def get_num_pages(state):
    current_page = 1
    while True:
        url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        next_page_button = soup.find_all('a', class_ = 'btn btn-default')
        button = [butt.text for butt in next_page_button]
        #print(button)
        num = str(current_page + 1)
        if num not in button:
            return int(num)- 1
        current_page += 1

In [ ]:
#this function will take in a state as an input and create a pandas dataframe with the school name, state rank, and SOS for each school in the state.
def extract_state_sos(state, current_page):
    url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text , 'html.parser')
    
    school_elements = soup.find_all('th', class_='school')
    school_elements.pop(0)
    school_links = [school.find('a').get('href') for school in school_elements]
    
    school_elements = soup.find_all('th', class_='school')
    school_elements.pop(0)
    school_names = [school.text for school in school_elements]

    school_rk_elements = soup.find_all('td', class_='rank first dw')
    school_rks = [rk.text for rk in school_rk_elements]

    school_sos_elements = soup.find_all('td', class_ = 'strength dw')
    school_sos = [sos.text for sos in school_sos_elements]

    school_rating_elements = soup.find_all('td', class_ = 'rating sorted dw')
    school_rating = [sos.text for sos in school_rating_elements]

    return pd.DataFrame({'School': school_names, 'State Rank': school_rks, 'SOS': school_sos, 'Rating': school_rating, 'State': state.upper(), 'URL': school_links})


In [ ]:
#function that will loop through all the pages of a state and extract the school name, state rank, and SOS for each school in the state.
def get_state_sos(state):
    state_df = pd.DataFrame()
    for i in tqdm(range(1, get_num_pages(state) + 1), desc=f'{state}',leave=False):
        page_data = extract_state_sos(state, i)
        state_df = pd.concat([state_df, page_data], ignore_index = True)

    return state_df


In [ ]:
states = [
    "ak", "al", "ar", "az", "ca", "co", "ct", "dc", "de", "fl", "ga",
    "hi", "ia", "id", "il", "in", "ks", "ky", "la", "ma", "md",
    "me", "mi", "mn", "mo", "ms", "mt", "nc", "nd", "ne", "nh",
    "nj", "nm", "nv", "ny", "oh", "ok", "or", "pa", "ri", "sc",
    "sd", "tn", "tx", "ut", "va", "vt", "wa", "wi", "wv", "wy"
]

usa_sos = pd.DataFrame()
for state in states:
    state_df = get_state_sos(state)
    usa_sos = pd.concat([usa_sos, state_df], ignore_index = True)

usa_sos


In [202]:
#merge on the sos, rating, state rank onto the final df
right = usa_sos[['SOS', 'Rating', 'State Rank', 'URL']]
df = pd.merge(all_states, right, left_on='Team 1 URL', right_on = 'URL', how='left').drop('URL', axis=1)
df

,Date,Team 1,Team 2,Venue,Game Type,Team 1 Score,Team 2 Score,Outcome,Team 1 Address,Team 1 City,Team 1 State,Team 1 Zipcode,Team 1 URL,Team 2 URL,Boys Sports,Girls Sports,SOS,Rating,State Rank
0,12/2,Academy of Holy Angels (Richfield),Edison,H,Regular Season,72.0,62.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/minneapolis/edison-tommies/basketball/21-2...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",9.0,3.64,180
1,12/7,Academy of Holy Angels (Richfield),Minneapolis Southwest,A,Regular Season,62.0,37.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/minneapolis/minneapolis-southwest-lakers/b...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",9.0,3.64,180
2,12/9,Academy of Holy Angels (Richfield),Highland Park,A,Regular Season,65.0,50.0,W,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/st-paul/highland-park-scots/basketball/21-...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",9.0,3.64,180
3,12/14,Academy of Holy Angels (Richfield),Two Rivers,H,Regular Season,74.0,69.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/mendota-heights/two-rivers-warriors/basket...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",9.0,3.64,180
4,12/16,Academy of Holy Angels (Richfield),St. Thomas,H,Regular Season,74.0,70.0,L,6600 Nicollet Ave S,Richfield,MN,55423.0,/mn/richfield/academy-of-holy-angels-stars/bas...,/mn/mendota-heights/st-thomas-academy-cadets/b...,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",9.0,3.64,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468312,2/10,White Sulphur Springs,West Yellowstone,H,District,77.0,54.0,L,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/yellowstone/west-yellowstone-wolverines/ba...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",-1.8,-5.34,94
468313,2/12,White Sulphur Springs,Shields Valley,H,District,91.0,59.0,L,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/clyde-park/shields-valley-rebels/basketbal...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",-1.8,-5.34,94
468314,2/16,White Sulphur Springs,Lone Peak,H,Playoff,71.0,60.0,W,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/gallatin-gateway/lone-peak-big-horns/baske...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",-1.8,-5.34,94
468315,2/17,White Sulphur Springs,Shields Valley,H,Playoff,74.0,48.0,L,405 Central Ave S,White Sulphur Springs,MT,59645.0,/mt/white-sulphur-springs/white-sulphur-spring...,/mt/clyde-park/shields-valley-rebels/basketbal...,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",-1.8,-5.34,94


### Replicate all information on the Team 2 Side

In [204]:
#we want to split the data into a part that includes the URL, and Team 1 location data
right = df[['Team 1 Address', 'Team 1 City', 'Team 1 State', 'Team 1 Zipcode', 'Boys Sports', 'Girls Sports', 'SOS', 'State Rank', 'Rating', 'Team 1 URL']].drop_duplicates()
right.rename(columns={'Team 1 Address': 'Team 2 Address', 'Team 1 City': 'Team 2 City', 'Team 1 State': 'Team 2 State', 'Team 1 Zipcode': 'Team 2 Zipcode', 'Team 1 URL': 'URL', 'Boys Sports' : 'Team 2 Boys Sports', 
                      'Girls Sports': 'Team 2 Girls Sports', 'State Rank': 'Team 2 State Rank', 'SOS': 'Team 2 SOS', 'Rating': 'Team 2 Rating'}, inplace=True)
right

,Team 2 Address,Team 2 City,Team 2 State,Team 2 Zipcode,Team 2 Boys Sports,Team 2 Girls Sports,Team 2 SOS,Team 2 State Rank,Team 2 Rating,URL
0,6600 Nicollet Ave S,Richfield,MN,55423.0,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",9.0,180,3.64,/mn/richfield/academy-of-holy-angels-stars/bas...
30,215 N Broadway,Alden,MN,56009.0,"Baseball,Basketball,Football,Wrestling","Basketball,Softball,Volleyball",-4.7,411,-19.45,/mn/alden/alden-conger-knights/basketball/21-2...
57,415 Kentucky Ave,Adrian,MN,56110.0,"Baseball,Basketball,Football,Wrestling","Basketball,Softball,Volleyball",1.3,382,-15.70,/mn/adrian/adrian-dragons/basketball/21-22/sch...
77,100 E Vadnais Blvd,Vadnais Heights,MN,55127.0,"Baseball,Basketball,Soccer,Wrestling","Basketball,Soccer,Softball,Volleyball",-19.6,424,-24.58,/mn/vadnais-heights/academy-for-sciences-and-a...
93,415 Kentucky Ave,Adrian,MN,56110.0,"Baseball,Basketball,Football","Basketball,Softball,Volleyball",-4.2,383,-16.28,/mn/adrian/adrian-ellsworth-dragons/basketball...
...,...,...,...,...,...,...,...,...,...,...
468234,1 West Yellowstone,Whitehall,MT,59759.0,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",8.4,128,-11.31,/mt/whitehall/whitehall-trojans/basketball/21-...
468247,121 F St. North,Wibaux,MT,59353.0,"Basketball,Football,Soccer,Wrestling","Basketball,Soccer,Softball,Volleyball",-2.0,118,-9.24,/mt/wibaux/wibaux-longhorns/basketball/21-22/s...
468265,213 6Th Ave S,Wolf Point,MT,59201.0,"Basketball,Football,Soccer,Wrestling","Basketball,Soccer,Softball,Volleyball",-0.6,64,3.70,/mt/wolf-point/wolf-point-wolves/basketball/21...
468279,1143 East 4th,Whitefish,MT,59937.0,"Baseball,Basketball,Football,Soccer,Wrestling","Basketball,Soccer,Softball,Volleyball",8.2,61,4.11,/mt/whitefish/whitefish-bulldogs/basketball/21...


In [205]:
#we want to merge it back into the final df to fill in the missing Team 2 location data
df = pd.merge(df, right, left_on='Team 2 URL', right_on = 'URL', how='left')
df

,Date,Team 1,Team 2,Venue,Game Type,Team 1 Score,Team 2 Score,Outcome,Team 1 Address,Team 1 City,...,Team 2 Address,Team 2 City,Team 2 State,Team 2 Zipcode,Team 2 Boys Sports,Team 2 Girls Sports,Team 2 SOS,Team 2 State Rank,Team 2 Rating,URL
0,12/2,Academy of Holy Angels (Richfield),Edison,H,Regular Season,72.0,62.0,L,6600 Nicollet Ave S,Richfield,...,700 NE 22Nd Ave,Minneapolis,MN,55418.0,"Baseball,Basketball,Football,Lacrosse,Soccer,V...","Basketball,Lacrosse,Soccer,Softball,Tennis,Vol...",5.0,189,2.46,/mn/minneapolis/edison-tommies/basketball/21-2...
1,12/7,Academy of Holy Angels (Richfield),Minneapolis Southwest,A,Regular Season,62.0,37.0,L,6600 Nicollet Ave S,Richfield,...,3414 W 47Th St,Minneapolis,MN,55410.0,"Baseball,Basketball,Football,Lacrosse,Soccer,V...","Basketball,Lacrosse,Soccer,Softball,Tennis,Vol...",8.6,86,10.99,/mn/minneapolis/minneapolis-southwest-lakers/b...
2,12/9,Academy of Holy Angels (Richfield),Highland Park,A,Regular Season,65.0,50.0,W,6600 Nicollet Ave S,Richfield,...,1015 Snelling Ave S,St. Paul,MN,55116.0,"Baseball,Basketball,Football,Ice Hockey,Soccer...","Basketball,Ice Hockey,Soccer,Softball,Tennis,V...",-0.3,287,-5.43,/mn/st-paul/highland-park-scots/basketball/21-...
3,12/14,Academy of Holy Angels (Richfield),Two Rivers,H,Regular Season,74.0,69.0,L,6600 Nicollet Ave S,Richfield,...,1897 Delaware Ave,Mendota Heights,MN,55118.0,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",4.3,129,7.36,/mn/mendota-heights/two-rivers-warriors/basket...
4,12/16,Academy of Holy Angels (Richfield),St. Thomas,H,Regular Season,74.0,70.0,L,6600 Nicollet Ave S,Richfield,...,949 Mendota Heights Rd,Mendota Heights,MN,55120.0,"Baseball,Basketball,Football,Ice Hockey,Lacros...",NaN,7.4,101,9.92,/mn/mendota-heights/st-thomas-academy-cadets/b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468312,2/10,White Sulphur Springs,West Yellowstone,H,District,77.0,54.0,L,405 Central Ave S,White Sulphur Springs,...,411 N. Geyser,Yellowstone,MT,59758.0,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",7.1,39,9.12,/mt/yellowstone/west-yellowstone-wolverines/ba...
468313,2/12,White Sulphur Springs,Shields Valley,H,District,91.0,59.0,L,405 Central Ave S,White Sulphur Springs,...,405 1st Street East,Clyde Park,MT,59018.0,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",3.2,42,8.75,/mt/clyde-park/shields-valley-rebels/basketbal...
468314,2/16,White Sulphur Springs,Lone Peak,H,Playoff,71.0,60.0,W,405 Central Ave S,White Sulphur Springs,...,45465 Gallatin Rd.,Gallatin Gateway,MT,59730.0,"Baseball,Basketball,Football,Soccer","Basketball,Volleyball",1.7,129,-11.35,/mt/gallatin-gateway/lone-peak-big-horns/baske...
468315,2/17,White Sulphur Springs,Shields Valley,H,Playoff,74.0,48.0,L,405 Central Ave S,White Sulphur Springs,...,405 1st Street East,Clyde Park,MT,59018.0,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",3.2,42,8.75,/mt/clyde-park/shields-valley-rebels/basketbal...


In [213]:
#rename and reorder the columns to final format
df = df.rename(columns={'Boys Sports': 'Team 1 Boys Sports', 'Girls Sports':'Team 1 Girls Sports', 'SOS': 'Team 1 SOS', 'Rating': 'Team 1 Rating', 'State Rank': 'Team 1 State Rank'})
cols=['Date', 'Team 1', 'Team 2', 'Venue', 'Game Type', 'Team 1 Score',
       'Team 2 Score', 'Outcome', 'Team 1 Address', 'Team 1 City',
       'Team 1 State', 'Team 1 Zipcode', 'Team 1 URL', 'Team 2 URL',
       'Team 1 Boys Sports', 'Team 1 Girls Sports', 'Team 1 SOS', 'Team 1 State Rank',
       'Team 1 Rating', 'Team 2 Address', 'Team 2 City',
       'Team 2 State', 'Team 2 Zipcode', 'Team 2 Boys Sports',
       'Team 2 Girls Sports', 'Team 2 SOS', 'Team 2 State Rank',
       'Team 2 Rating']
df = df[cols]
df

,Date,Team 1,Team 2,Venue,Game Type,Team 1 Score,Team 2 Score,Outcome,Team 1 Address,Team 1 City,...,Team 1 Rating,Team 2 Address,Team 2 City,Team 2 State,Team 2 Zipcode,Team 2 Boys Sports,Team 2 Girls Sports,Team 2 SOS,Team 2 State Rank,Team 2 Rating
0,12/2,Academy of Holy Angels (Richfield),Edison,H,Regular Season,72.0,62.0,L,6600 Nicollet Ave S,Richfield,...,3.64,700 NE 22Nd Ave,Minneapolis,MN,55418.0,"Baseball,Basketball,Football,Lacrosse,Soccer,V...","Basketball,Lacrosse,Soccer,Softball,Tennis,Vol...",5.0,189,2.46
1,12/7,Academy of Holy Angels (Richfield),Minneapolis Southwest,A,Regular Season,62.0,37.0,L,6600 Nicollet Ave S,Richfield,...,3.64,3414 W 47Th St,Minneapolis,MN,55410.0,"Baseball,Basketball,Football,Lacrosse,Soccer,V...","Basketball,Lacrosse,Soccer,Softball,Tennis,Vol...",8.6,86,10.99
2,12/9,Academy of Holy Angels (Richfield),Highland Park,A,Regular Season,65.0,50.0,W,6600 Nicollet Ave S,Richfield,...,3.64,1015 Snelling Ave S,St. Paul,MN,55116.0,"Baseball,Basketball,Football,Ice Hockey,Soccer...","Basketball,Ice Hockey,Soccer,Softball,Tennis,V...",-0.3,287,-5.43
3,12/14,Academy of Holy Angels (Richfield),Two Rivers,H,Regular Season,74.0,69.0,L,6600 Nicollet Ave S,Richfield,...,3.64,1897 Delaware Ave,Mendota Heights,MN,55118.0,"Baseball,Basketball,Football,Ice Hockey,Lacros...","Basketball,Ice Hockey,Lacrosse,Soccer,Softball...",4.3,129,7.36
4,12/16,Academy of Holy Angels (Richfield),St. Thomas,H,Regular Season,74.0,70.0,L,6600 Nicollet Ave S,Richfield,...,3.64,949 Mendota Heights Rd,Mendota Heights,MN,55120.0,"Baseball,Basketball,Football,Ice Hockey,Lacros...",NaN,7.4,101,9.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468312,2/10,White Sulphur Springs,West Yellowstone,H,District,77.0,54.0,L,405 Central Ave S,White Sulphur Springs,...,-5.34,411 N. Geyser,Yellowstone,MT,59758.0,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",7.1,39,9.12
468313,2/12,White Sulphur Springs,Shields Valley,H,District,91.0,59.0,L,405 Central Ave S,White Sulphur Springs,...,-5.34,405 1st Street East,Clyde Park,MT,59018.0,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",3.2,42,8.75
468314,2/16,White Sulphur Springs,Lone Peak,H,Playoff,71.0,60.0,W,405 Central Ave S,White Sulphur Springs,...,-5.34,45465 Gallatin Rd.,Gallatin Gateway,MT,59730.0,"Baseball,Basketball,Football,Soccer","Basketball,Volleyball",1.7,129,-11.35
468315,2/17,White Sulphur Springs,Shields Valley,H,Playoff,74.0,48.0,L,405 Central Ave S,White Sulphur Springs,...,-5.34,405 1st Street East,Clyde Park,MT,59018.0,"Basketball,Football,Soccer,Track & Field,Wrest...","Basketball,Soccer,Softball,Volleyball",3.2,42,8.75


In [217]:
#function that will remove the city in parenthesis next to the school name.
def remove_city(school):
    return school.split('(')[0].strip()

In [226]:
#format the typos created in maxpreps for the teams that do not have mascots
def fixName(string):
    if len(string) > 2:  
        if string[0] == string[1] and string[0].isupper():
            string = string[1:]
    return string

In [227]:
df['Team 1'] = df['Team 1'].apply(remove_city)
df['Team 2'] = df['Team 2'].apply(fixName)

In [214]:
# Assuming df is your DataFrame
null_counts = df.isnull().sum()

print(null_counts)

Date                       0
Team 1                     0
Team 2                     0
Venue                      0
Game Type                  0
Team 1 Score           22176
Team 2 Score           22176
Outcome                22176
Team 1 Address          5106
Team 1 City                0
Team 1 State               0
Team 1 Zipcode          5038
Team 1 URL                 0
Team 2 URL              1210
Team 1 Boys Sports         0
Team 1 Girls Sports     3983
Team 1 SOS                 0
Team 1 State Rank          0
Team 1 Rating              0
Team 2 Address         13677
Team 2 City             8947
Team 2 State            8947
Team 2 Zipcode         13652
Team 2 Boys Sports      8947
Team 2 Girls Sports    12701
Team 2 SOS              8947
Team 2 State Rank       8947
Team 2 Rating           8947
dtype: int64


In [229]:
os.getcwd()
os.chdir('/Users/rdhir/Documents/maxpreps-data-analysis/Data')

In [230]:
df.to_csv('maxpreps.csv', index=False)